###Lanczos Interpolation

In [2]:
import numpy as np
import cv2

def lanczos_interp(image, scale_factor):
    # Define Lanczos kernel size and radius
    k_size = 8
    radius = 3

    # Generate 1D Lanczos kernel
    x = np.linspace(0, radius, k_size, endpoint=False) * np.pi
    lanczos_1d = np.sin(x) / x
    lanczos_1d[x == 0] = 1

    # Create 2D Lanczos kernel
    lanczos_2d = np.outer(lanczos_1d, lanczos_1d)

    # Normalize kernel to have unit sum
    lanczos_2d /= lanczos_2d.sum()

    # Upscale image using Lanczos interpolation
    height, width = image.shape[:2]
    new_height, new_width = int(scale_factor * height), int(scale_factor * width)
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)

    # Resample using Lanczos kernel
    output_image = cv2.filter2D(resized_image, -1, lanczos_2d)

    return output_image

# Example usage
input_image = cv2.imread('coffee.jpeg')
output_image = lanczos_interp(input_image, 2.0)
cv2.imwrite('output_image.jpg', output_image)


<ipython-input-2-fad71b0966a3>:11: RuntimeWarning: invalid value encountered in true_divide
  lanczos_1d = np.sin(x) / x


True

###Bicubic & Lanczos Interpolations Combined

In [1]:
from PIL import Image
import cv2
import numpy as np

def upscale_image(image_path, scale_factor):
    # Open image file
    image = Image.open(image_path)

    # Compute new size of image
    width, height = image.size
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)

    # Resize image using Lanczos interpolation
    lanczos_image = image.resize((new_width, new_height), resample=Image.LANCZOS)

    # Resize image using Bicubic interpolation
    bicubic_image = image.resize((new_width, new_height), resample=Image.BICUBIC)

    # Convert PIL image to OpenCV image
    lanczos_image_cv2 = cv2.cvtColor(np.array(lanczos_image), cv2.COLOR_RGB2BGR)
    bicubic_image_cv2 = cv2.cvtColor(np.array(bicubic_image), cv2.COLOR_RGB2BGR)

    # Apply denoising and sharpening to Lanczos image
    lanczos_image_cv2 = cv2.fastNlMeansDenoisingColored(lanczos_image_cv2, None, 10, 10, 7, 21)
    lanczos_image_cv2 = cv2.GaussianBlur(lanczos_image_cv2, (0,0), 1.5)
    lanczos_image_cv2 = cv2.addWeighted(lanczos_image_cv2, 1.5, cv2.GaussianBlur(lanczos_image_cv2, (0,0), 10), -0.5, 0)
    lanczos_image_cv2 = cv2.cvtColor(lanczos_image_cv2, cv2.COLOR_BGR2RGB)

    # Apply denoising and sharpening to Bicubic image
    bicubic_image_cv2 = cv2.fastNlMeansDenoisingColored(bicubic_image_cv2, None, 10, 10, 7, 21)
    bicubic_image_cv2 = cv2.GaussianBlur(bicubic_image_cv2, (0,0), 1.5)
    bicubic_image_cv2 = cv2.addWeighted(bicubic_image_cv2, 1.5, cv2.GaussianBlur(bicubic_image_cv2, (0,0), 10), -0.5, 0)
    bicubic_image_cv2 = cv2.cvtColor(bicubic_image_cv2, cv2.COLOR_BGR2RGB)

    # Combine Lanczos and Bicubic images
    combined_image_cv2 = cv2.addWeighted(lanczos_image_cv2, 0.5, bicubic_image_cv2, 0.5, 0)

    # Convert OpenCV image to PIL image
    combined_image_pil = Image.fromarray(combined_image_cv2)

    return combined_image_pil

upscaled_image = upscale_image('input_image.jpeg', 2)
upscaled_image.save('output_image.jpg')